In [1]:
import numpy as np
from sklearn.datasets import fetch_california_housing
import tensorflow as tf

In [2]:
housing = fetch_california_housing()
m, n = housing.data.shape

In [3]:
housing_data_plus_bias = np.c_[np.ones((m, 1)), housing.data]
housing_data_plus_bias.shape

(20640, 9)

In [4]:
X = tf.constant(housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")

In [5]:
XT = tf.transpose(X)
theta = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT, X)), XT), y)

In [6]:
with tf.Session() as sess:
    theta_value = theta.eval()

In [8]:
theta_value

array([[-3.6959320e+01],
       [ 4.3698898e-01],
       [ 9.4245886e-03],
       [-1.0791138e-01],
       [ 6.4842808e-01],
       [-3.9986235e-06],
       [-3.7866351e-03],
       [-4.2142656e-01],
       [-4.3467718e-01]], dtype=float32)

# Gradient Descent

In [11]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaled_housing_data_plus_bias = scaler.fit_transform(housing_data_plus_bias)

In [22]:
n_epochs = 1000
learning_rate = 0.01

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")
# gradients = 2/m * tf.matmul(tf.transpose(X), error) ## Manual calculation of gradients

gradients = tf.gradients(mse, [theta])[0] ## Calculating gradients with tensorflow's autodiff

training_op = tf.assign(theta, theta - learning_rate * gradients)

init = tf.global_variables_initializer()

In [23]:
with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch", epoch, "MSE=", mse.eval())
        sess.run(training_op)
        
    best_theta = theta.eval()

Epoch 0 MSE= 5.554215
Epoch 100 MSE= 4.923764
Epoch 200 MSE= 4.8863015
Epoch 300 MSE= 4.867178
Epoch 400 MSE= 4.8530955
Epoch 500 MSE= 4.84234
Epoch 600 MSE= 4.8340607
Epoch 700 MSE= 4.827654
Epoch 800 MSE= 4.8226686
Epoch 900 MSE= 4.8187695


In [17]:
best_theta

array([[-0.3689556 ],
       [ 0.9620415 ],
       [ 0.16135263],
       [-0.4824726 ],
       [ 0.47047076],
       [ 0.00927473],
       [-0.04575322],
       [-0.4649752 ],
       [-0.44943455]], dtype=float32)